# 🚀 Setup Instructions

1. **Install dependencies:**

In [ ]:
#Install required packages:
#!python.exe -m pip install --upgrade pip

%pip install gradio
%pip install requests
%pip install pyserial
%pip install ipywidgets
%pip install tqdm

#OPTIONal Upgrades:
# %pip install -U gradio
# %pip install -U jupyter ipywidgets

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# ⚙️ How to Use

1. **Enter your API keys** into the designated fields in the interface.
API Key Reference Sheet:  
https://docs.google.com/spreadsheets/d/1uUToB838M1xZ-G7FPOAUmK8ncmzgow_kfJplaghjAnA/edit?gid=0#gid=0

2. Make sure your environment is ready (e.g., USB connected if using Arduino).

3. **Write a prompt** such as:
   - “Generate a waveform with frequency and amplitude.”
   - “Suggest an Arduino side project.”
   - “Produce parameters for PWM waveform output.”

4. **Run the program** to get model-generated parameters (frequency, amplitude, waveform type).

5. Use the output for:
   - Arduino waveform generation (via Serial)
   - Side-project recommendations
   - Any other custom logic you’ve added

In [1]:
# File: openrouter_gen.py
# OpenRouter Frequency & Amplitude Generator
# note: Only UI/design changes were applied in this version. Functionality, logic, and handlers are unchanged.
# - Fixed gradio 'scale' warning (scale must be integer)
# - Polished button hover/active styles, transitions, pointer cursors
# - Kept logic and handlers unchanged

import json
import re
import socket
import urllib.parse
import os
from typing import Tuple, Dict, Any, List
import requests
import gradio as gr

import serial
import serial.tools.list_ports
import time
import threading

# Defaults
DEFAULT_OPENROUTER_URL = "https://openrouter.ai/api/v1/chat/completions"
DEFAULT_MODEL_UI = "x-ai/grok-4.1-fast:free"

VARS_FILE_PATH = "./last_vars.json"

# Module-level globals (set after a successful parse; available only in this Python process)
x = None  # Hz (int)
y = None  # Volts (float)

# Cancellation events
gen_cancel_event = threading.Event()
send_cancel_event = threading.Event()


# -------------------------
# (unchanged) Core helpers
# -------------------------
def build_instruction(user_prompt: str) -> str:
    return (
        "You are an assistant that MUST respond with a concise machine-friendly result.\n\n"
        "User prompt: " + user_prompt + "\n\n"
        "Also: create specific frequency and amplitude based on what you think we want most.\n\n"
        "OUTPUT REQUIREMENT (MANDATORY):\n"
        "- Return ONLY a single JSON object (no surrounding text) with exactly these keys:\n"
        '  frequency_hz : float frequency in Hertz (e.g. 1000.0)\n'
        '  amplitude_v  : float amplitude in Volts (e.g. 3.0)\n'
        '  x : duplicate of frequency_hz (float)\n'
        '  y : duplicate of amplitude_v (float)\n\n'
        "Example output (for one thousand Hz and three volts):\n"
        '{"frequency_hz": 1000.0, "amplitude_v": 3.0, "x": 1000.0, "y": 3.0}\n\n'
        "If you cannot infer exact numbers, pick reasonable defaults and set them.\n"
        "Do not include any extra commentary, explanation, or text outside the JSON object."
    )


def send_to_openrouter(api_key: str, endpoint: str, system_msg: str, user_msg: str, model: str) -> Dict[str, Any]:
    headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}
    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": system_msg},
            {"role": "user", "content": user_msg},
        ],
        "temperature": 0.2,
        "max_tokens": 250,
        "top_p": 0.9,
    }

    try:
        resp = requests.post(endpoint, headers=headers, json=payload, timeout=30)
    except requests.RequestException as e:
        raise RuntimeError(f"Connection error while contacting {endpoint}: {e}")

    if not resp.ok:
        try:
            body = resp.json()
        except Exception:
            body = resp.text
        raise RuntimeError(f"HTTP {resp.status_code} from {endpoint}: {body}")

    try:
        return resp.json()
    except Exception as e:
        raise RuntimeError(f"Failed to parse JSON response from {endpoint}: {e} -- raw: {resp.text}")


def extract_assistant_text_from_response(resp_json: Dict[str, Any]) -> str:
    if not isinstance(resp_json, dict):
        return json.dumps(resp_json)

    choices = resp_json.get("choices")
    if choices and isinstance(choices, list) and len(choices) > 0:
        first = choices[0]
        if isinstance(first.get("message"), dict):
            content = first["message"].get("content")
            if isinstance(content, str):
                return content
            if isinstance(content, dict) and "parts" in content:
                parts = content.get("parts")
                if isinstance(parts, list):
                    return "".join(parts)
        if "text" in first and isinstance(first["text"], str):
            return first["text"]

    for k in ("output", "text", "response", "content"):
        if k in resp_json and isinstance(resp_json[k], str):
            return resp_json[k]

    return json.dumps(resp_json, indent=2)


def parse_response_for_vars(text: str) -> Dict[str, Any]:
    text_stripped = (text or "").strip()

    # Try JSON extraction first
    json_obj = None
    try:
        json_obj = json.loads(text_stripped)
    except Exception:
        m = re.search(r"\{[\s\S]*?\}", text_stripped)
        if m:
            try:
                json_obj = json.loads(m.group(0))
            except Exception:
                json_obj = None

    frequency_hz = None
    amplitude_v = None

    if isinstance(json_obj, dict):
        for key in ["frequency_hz", "frequency", "freq_hz", "freq", "x"]:
            if key in json_obj:
                try:
                    frequency_hz = float(json_obj[key])
                    break
                except Exception:
                    pass
        for key in ["amplitude_v", "amplitude", "amp_v", "amp", "y"]:
            if key in json_obj:
                try:
                    amplitude_v = float(json_obj[key])
                    break
                except Exception:
                    pass

    # Regex heuristics if JSON parsing didn't work
    if frequency_hz is None:
        m = re.search(r"([0-9]+(?:\.[0-9]+)?)\s*(k|K)?\s*(?:hz|Hz|HZ)?\b", text_stripped)
        if m:
            try:
                num = float(m.group(1))
                mult = 1000.0 if (m.group(2) and m.group(2).lower() == "k") else 1.0
                frequency_hz = float(num * mult)
            except Exception:
                pass

    if amplitude_v is None:
        m = re.search(r"([0-9]+(?:\.[0-9]+)?)\s*(m?V|v|V)\b", text_stripped)
        if m:
            try:
                val = float(m.group(1))
                unit = m.group(2)
                amplitude_v = val / 1000.0 if unit.lower().startswith("mv") else float(val)
            except Exception:
                pass

    if frequency_hz is None:
        m = re.search(r"x\s*=\s*([0-9]+(?:\.[0-9]+)?)", text_stripped)
        if m:
            try:
                frequency_hz = float(m.group(1))
            except Exception:
                pass

    if amplitude_v is None:
        m = re.search(r"y\s*=\s*([0-9]+(?:\.[0-9]+)?)", text_stripped)
        if m:
            try:
                amplitude_v = float(m.group(1))
            except Exception:
                pass

    # Basic validation
    if isinstance(frequency_hz, (int, float)):
        frequency_hz = float(frequency_hz)
        if frequency_hz <= 0 or frequency_hz > 10_000_000:
            frequency_hz = None
        else:
            frequency_hz = float(round(frequency_hz, 6))
    if isinstance(amplitude_v, (int, float)):
        amplitude_v = float(amplitude_v)
        if amplitude_v <= 0 or amplitude_v > 1e6:
            amplitude_v = None
        else:
            amplitude_v = float(round(amplitude_v, 6))

    return {"frequency_hz": frequency_hz, "amplitude_v": amplitude_v}


def persist_and_set_vars(parsed: Dict[str, Any], persist_path: str = VARS_FILE_PATH) -> str:
    global x, y
    f = parsed.get("frequency_hz")
    a = parsed.get("amplitude_v")
    lines = []

    if f is None or a is None:
        lines.append("Parsing incomplete: not setting globals/environment variables, but will still persist whatever was parsed.")
    else:
        x = float(f)
        y = float(a)
        lines.append(f"Set in-process globals: x = {x} (Hz), y = {y} (V)")
        os.environ["FREQUENCY_HZ"] = str(x)
        os.environ["AMPLITUDE_V"] = str(y)
        lines.append("Set environment variables: FREQUENCY_HZ, AMPLITUDE_V")

    try:
        data = {"frequency_hz": f, "amplitude_v": a, "x": f, "y": a}
        with open(persist_path, "w") as fh:
            json.dump(data, fh)
        lines.append(f"Wrote variables to {persist_path}")
    except Exception as e:
        lines.append(f"Failed to write variables to {persist_path}: {e}")

    return "\n".join(lines)


def generate_and_parse(api_key: str, endpoint: str, model: str, prompt: str, cancel_event: threading.Event = None) -> Tuple[str, Dict[str, Any], str, str]:
    if cancel_event is None:
        cancel_event = threading.Event()

    if not api_key:
        return "ERROR: Missing API key (do not paste keys into public notebooks).", {}, "", ""

    if cancel_event.is_set():
        return "CANCELED: Generation was canceled before starting.", {}, "", "Generation canceled by user."

    system_instruction = build_instruction(prompt)
    try:
        resp_json = send_to_openrouter(api_key=api_key, endpoint=endpoint, system_msg=system_instruction, user_msg=prompt, model=model)
    except RuntimeError as e:
        if cancel_event.is_set():
            return "CANCELED: Generation canceled.", {}, "", "Generation canceled by user."
        err_msg = str(e)
        suggestion = ""
        if "is not a valid model" in err_msg or '"is not a valid model' in err_msg:
            suggestion = "\n\nDetected server-side invalid-model error. Try a different model ID."
        return f"ERROR contacting OpenRouter: {err_msg}{suggestion}", {}, "", ""

    if cancel_event.is_set():
        return "CANCELED: Generation canceled after response.", {}, "", "Generation canceled by user."

    assistant_text = extract_assistant_text_from_response(resp_json)
    parsed = parse_response_for_vars(assistant_text)

    f = parsed.get("frequency_hz")
    a = parsed.get("amplitude_v")

    if f is not None and a is not None:
        code_snippet = (
            f"// Standard variables\n"
            f"frequency_hz = {float(round(f,6))}  # Hz\n"
            f"amplitude_v = {float(round(a,6))}  # volts\n"
            f"x = {float(round(f,6))}\n"
            f"y = {float(round(a,6))}\n"
        )
    else:
        code_snippet = "# Could not parse both frequency and amplitude from model output.\n# See raw response above."

    status = persist_and_set_vars(parsed, persist_path=VARS_FILE_PATH)

    return assistant_text, parsed, code_snippet, status


def try_model_list_endpoint(api_key: str, url: str) -> Tuple[bool, Any]:
    headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}
    try:
        r = requests.get(url, headers=headers, timeout=15)
    except requests.RequestException as e:
        return False, f"Connection error: {e}"

    if not r.ok:
        try:
            return False, r.json()
        except Exception:
            return False, r.text

    try:
        return True, r.json()
    except Exception as e:
        return False, f"Failed to parse JSON: {e} -- raw: {r.text}"


def list_models(api_key: str, endpoint: str) -> Tuple[str, List[str]]:
    if not api_key:
        return "ERROR: Missing API key.", []

    try:
        parsed = urllib.parse.urlparse(endpoint)
        base = f"{parsed.scheme}://{parsed.netloc}"
    except Exception:
        base = endpoint.rstrip("/")

    candidate_paths = [
        f"{base}/api/v1/models",
        f"{base}/api/v1/models?include_all=true",
        f"{base}/api/models",
        f"{base}/v1/models",
        f"{base}/api/v1/engines",
        f"{base}/v1/engines",
    ]

    diagnostics = []
    found_models = set()

    for url in candidate_paths:
        diagnostics.append(f"Trying: {url}")
        ok, result = try_model_list_endpoint(api_key, url)
        if not ok:
            diagnostics.append(f"  -> failed: {result}")
            continue

        diagnostics.append(f"  -> success: got JSON (attempting to extract model ids)")
        try:
            if isinstance(result, dict):
                if "models" in result and isinstance(result["models"], list):
                    for m in result["models"]:
                        if isinstance(m, dict):
                            if "id" in m:
                                found_models.add(str(m["id"]))
                            elif "model" in m:
                                found_models.add(str(m["model"]))
                            elif "name" in m:
                                found_models.add(str(m["name"]))
                            else:
                                found_models.add(json.dumps(m))
                        else:
                            found_models.add(str(m))
                elif "data" in result and isinstance(result["data"], list):
                    for m in result["data"]:
                        if isinstance(m, dict):
                            if "id" in m:
                                found_models.add(str(m["id"]))
                            elif "model" in m:
                                found_models.add(str(m["model"]))
                            elif "name" in m:
                                found_models.add(str(m["name"]))
                            else:
                                found_models.add(json.dumps(m))
                        else:
                            found_models.add(str(m))
                else:
                    text = json.dumps(result)
                    candidates = re.findall(r'"([a-zA-Z0-9_\-\.]{3,60})"', text)
                    for c in candidates:
                        if len(c) > 3 and not c.lower().startswith("http") and c.lower() not in ("true", "false", "null"):
                            found_models.add(c)
            else:
                diagnostics.append(f"  -> unexpected response type: {type(result)}")
        except Exception as e:
            diagnostics.append(f"  -> extraction error: {e}")
        if found_models:
            break

    diag_text = "\n".join(diagnostics)
    return diag_text, sorted(found_models)


def list_all_ports_text() -> str:
    ports = serial.tools.list_ports.comports()
    if not ports:
        return "No serial ports found"
    lines = []
    for p in ports:
        lines.append(f"{p.device}  - {p.description}")
    return "\n".join(lines)


def send_parsed_to_arduino(parsed: Dict[str, Any], port: str, baudrate: int = 115200, timeout: float = 5.0) -> str:
    global send_cancel_event

    if send_cancel_event.is_set():
        return "CANCELED: send-to-Arduino canceled by user."

    if not port:
        return "ERROR: No port provided."
    if not isinstance(parsed, dict):
        return "ERROR: No parsed variables available."

    f = parsed.get("frequency_hz")
    a = parsed.get("amplitude_v")
    if f is None or a is None:
        return "ERROR: Parsed frequency or amplitude is missing."

    try:
        ser = serial.Serial(port=port, baudrate=baudrate, timeout=0.1)
    except Exception as e:
        return f"ERROR opening {port}: {e}"

    start = time.time()
    got_ready = False
    ready_text = ""
    try:
        while time.time() - start < timeout:
            if send_cancel_event.is_set():
                return "CANCELED: send-to-Arduino canceled by user (during wait for ARDUINO_READY)."
            try:
                line = ser.readline().decode(errors='ignore').strip()
            except Exception:
                line = ""
            if not line:
                time.sleep(0.05)
                continue
            ready_text += line + "\n"
            if "ARDUINO_READY" in line:
                got_ready = True
                break
        if not got_ready:
            ready_msg = f"WARNING: did not see ARDUINO_READY within {timeout}s. Received:\n{ready_text}"
        else:
            ready_msg = f"Found ARDUINO_READY. Received:\n{ready_text}"

        if send_cancel_event.is_set():
            return "CANCELED: send-to-Arduino canceled by user before sending."

        send_line = f"{float(f)} {float(a)}\n"
        ser.write(send_line.encode())
        ser.flush()

        ack_start = time.time()
        ack = None
        ack_text = ""
        while time.time() - ack_start < timeout:
            if send_cancel_event.is_set():
                return "CANCELED: send-to-Arduino canceled by user (waiting for ACK)."
            try:
                line = ser.readline().decode(errors='ignore').strip()
            except Exception:
                line = ""
            if not line:
                time.sleep(0.02)
                continue
            ack_text += line + "\n"
            if line == "ACK":
                ack = "ACK"
                break
            if line == "NACK" or "PARSE_ERROR" in line:
                ack = "NACK"
                break

        if ack == "ACK":
            extra = ""
            extra_start = time.time()
            while time.time() - extra_start < 0.2:
                if send_cancel_event.is_set():
                    return "CANCELED: send-to-Arduino canceled by user (after ACK)."
                try:
                    line = ser.readline().decode(errors='ignore').strip()
                except Exception:
                    line = ""
                if line:
                    extra += line + "\n"
            return f"SUCCESS: Sent {send_line.strip()}\nHost ready check: {ready_msg}\nDevice response:\n{ack_text}{extra}"
        else:
            return f"ERROR: No ACK received. Host ready check: {ready_msg}\nDevice response:\n{ack_text}"
    except Exception as e:
        return f"ERROR during comms: {e}"
    finally:
        try:
            ser.close()
        except Exception:
            pass


def build_manual_parsed(hz: Any, volt: Any) -> Dict[str, Any]:
    try:
        f = None if hz is None else float(round(float(hz), 6))
    except Exception:
        f = None
    try:
        a = None if volt is None else float(round(float(volt), 6))
    except Exception:
        a = None
    return {"frequency_hz": f, "amplitude_v": a}


def manual_parsed_to_code(parsed: Dict[str, Any]) -> str:
    f = parsed.get("frequency_hz")
    a = parsed.get("amplitude_v")
    if f is not None and a is not None:
        return (
            f"// Manual variables\n"
            f"frequency_hz = {float(round(f, 6))}  # integer Hz\n"
            f"amplitude_v = {float(round(a,6))}  # volts\n"
            f"x = {float(round(f,6))}\n"
            f"y = {float(round(a,6))}\n"
        )
    return "# Manual input incomplete: provide both frequency and amplitude."


# -------------------------
# UI: DESIGN-ONLY CHANGES
# - Sleek, website-like look
# - Responsive two-column layout (left content area, right control panel)
# - Custom CSS via an injected <style> tag
# - Fixed scale usage (integers only)
# - Button hover & active interactions added via CSS (no functionality changes)
# -------------------------
with gr.Blocks(title="OpenRouter Frequency & Amplitude Generator") as demo:
    # Inject custom CSS for a polished, website-like appearance.
    gr.HTML(
        """
        <style>
        /* Page font + subtle background */
        :root{
            --bg:#0f1724;
            --card:#0b1220;
            --muted:#9aa4b2;
            --accent:#0b5fff;
            --accent-2:#7b61ff;
            --glass: rgba(255,255,255,0.03);
            --success: #22c55e;
            --danger: #ff6b6b;
        }
        body { background: linear-gradient(180deg, #071024 0%, #08172b 100%); font-family: Inter, ui-sans-serif, system-ui, -apple-system, "Segoe UI", Roboto, "Helvetica Neue", Arial; color: #e6eef8; }
        .gradio-container { max-width:1200px; margin: 18px auto; padding: 20px; }
        .app-header { text-align:center; margin-bottom: 18px; }
        .app-title { font-size:28px; font-weight:700; letter-spacing:-0.5px; color:var(--accent); margin:0; }
        .app-sub { font-size:13px; color:var(--muted); margin-top:6px; }

        /* Card styles */
        .card { background: linear-gradient(180deg, rgba(255,255,255,0.02), rgba(255,255,255,0.01)); border-radius:12px; padding:14px; box-shadow: 0 6px 20px rgba(2,6,23,0.6); border: 1px solid rgba(255,255,255,0.03); }
        .panel { padding:8px; }

        /* Two-column layout */
        .main-row { display:flex; gap:18px; align-items:flex-start; }
        .left-col { flex: 2 1 0; min-width: 480px; }
        .right-col { flex: 1 1 320px; min-width: 320px; }

        /* Headline within panel */
        .panel h3 { margin:0 0 8px 0; color:#d8e7ff; }

/* Button styles (gradio buttons still function) */
.gr-button {
    position: relative;
    overflow: hidden !important;
    border-radius: 12px !important;
    padding: 12px 20px !important;
    font-weight: 700;
    cursor: pointer !important;
    color: white !important;
    border: none !important;
    background: linear-gradient(145deg, #0b5fff, #7b61ff); /* base gradient */
    box-shadow: 0 6px 20px rgba(2,6,23,0.6);
    transition: all 0.35s ease;
    text-transform: uppercase;
    letter-spacing: 0.5px;
}

/* Hover effect: metallic gradient + shine */
.gr-button:hover {
    background: linear-gradient(145deg, #7b61ff, #0b5fff, #22c55e);
    box-shadow: 0 12px 30px rgba(11,95,255,0.3), 0 0 8px rgba(255,255,255,0.2) inset;
    transform: translateY(-2px) scale(1.03);
}

/* Active press effect */
.gr-button:active {
    transform: translateY(0px) scale(0.98);
    box-shadow: 0 4px 14px rgba(11,95,255,0.25) inset;
}

/* Primary button override (optional: stronger metallic) */
.primary {
    background: linear-gradient(145deg, #0b5fff, #7b61ff);
}

/* Secondary button override (subtle metallic) */
.secondary {
    background: linear-gradient(145deg, rgba(255,255,255,0.05), rgba(255,255,255,0.01));
    color: var(--muted) !important;
    border: 1px solid rgba(255,255,255,0.08) !important;
}



        /* Text areas sizing */
        .raw-output { min-height:160px; max-height:420px; overflow:auto; }
        .code-output { min-height:80px; font-family: ui-monospace, SFMono-Regular, Menlo, Monaco, "Roboto Mono", monospace; font-size:12.5px; background:rgba(255,255,255,0.02); border-radius:8px; padding:10px; color:#e6eef8; }

        /* Compact rows */
        .compact-row { display:flex; gap:10px; align-items:center; margin-top:8px; }

        /* Form inputs full width inside cards */
        .card .gr-textbox, .card .gr-number, .card .gr-dropdown, .card .gr-json {
            width:100% !important;
        }

        /* Small helper text */
        .helper { font-size:12px; color:var(--muted); margin-top:6px; }

        /* Make JSON area visually match code box on older gradio versions */
        .gr-json { background: linear-gradient(180deg, rgba(255,255,255,0.01), rgba(255,255,255,0.005)); border-radius:8px; padding:8px; color:#dbeafe; }

        /* Responsive for narrow screens */
        @media (max-width: 900px) {
            .main-row { flex-direction: column; }
            .left-col, .right-col { min-width: auto; width:100%; }
        }
        </style>
        """
    )

    # Header
    with gr.Row(elem_id="header", visible=True):
        with gr.Column():
            gr.HTML(
                """
                <div class="app-header">
                  <h1 class="app-title">OpenRouter Frequency & Amplitude Generator</h1>
                  <div class="app-sub">AI-assisted frequency & amplitude suggestions — manual overrides & Arduino send built-in</div>
                </div>
                """
            )

    # Top credentials card (use gr.Column with elem_classes="card" for compatibility)
    with gr.Row():
        with gr.Column():
            with gr.Row():
                with gr.Column():
                    # FIX: scale must be integer -> use 1 instead of 0.5 (avoids UserWarning)
                    with gr.Column(elem_classes="card"):
                        with gr.Row():
                            with gr.Column():
                                api_key_input = gr.Textbox(label="OpenRouter API Key", type="password",
                                                          placeholder="Paste API key (kept only in memory)")
                                endpoint_input = gr.Textbox(label="Endpoint URL", value=DEFAULT_OPENROUTER_URL)
                                model_input = gr.Textbox(label="Model (optional)", value=DEFAULT_MODEL_UI)
                            # use scale=1 (integer) to avoid gradio warning; CSS controls width/appearance
                            with gr.Column(scale=1):
                                # Action buttons grouped vertically to the right
                                with gr.Column(elem_classes="panel"):
                                    gen_button = gr.Button("Generate from AI", elem_id="gen-btn", variant="primary")
                                    cancel_gen_button = gr.Button("Cancel generation", elem_id="cancel-gen-btn", variant="secondary")
                                    list_ports_button = gr.Button("List serial ports", elem_id="list-ports-btn", variant="secondary")
                                    send_to_arduino_button = gr.Button("Send to Arduino", elem_id="send-arduino-btn", variant="primary")
                                    cancel_send_button = gr.Button("Cancel send", elem_id="cancel-send-btn", variant="secondary")
                                    # Small helper under buttons
                                    gr.HTML('<div class="helper">Use cancel buttons if an operation appears stuck. Network/serial operations are cooperative cancellations.</div>')

    gr.Markdown("<div style='height:6px'></div>")

    # Main content two-column layout
    with gr.Row(elem_classes="main-row"):
        # Left: prompt and outputs
        with gr.Column(elem_classes="left-col"):
            with gr.Column(elem_classes="card"):
                gr.HTML("<h3>Prompt & AI Generation</h3>")
                prompt_input = gr.Textbox(
                    label="Prompt",
                    placeholder="e.g. Input recommended Hertz and Voltage for Arduino Nano 33 BLE waveform generator",
                    lines=4,
                )
                with gr.Row(elem_classes="compact-row"):
                    gr.HTML("")  # spacer to align
                    # generation controls already provided above (buttons) - kept for design
                gr.HTML("<h3 style='margin-top:12px'>Model output</h3>")
                raw_out = gr.Textbox(label="Raw model response", lines=8, elem_classes="raw-output")
                parsed_out = gr.JSON(label="Parsed variables")
                code_out = gr.Textbox(label="Code snippet", lines=6, elem_classes="code-output")
                status_out = gr.Textbox(label="Status (persistence / env / in-process / cancellation)", lines=4)

        # Right: manual inputs, serial, quick actions
        with gr.Column(elem_classes="right-col"):
            with gr.Column(elem_classes="card"):
                gr.HTML("<h3>Quick Controls</h3>")
                gr.HTML("<div class='helper'>Manual overrides — apply values directly to process/environment and send to Arduino.</div>")
                hz_input = gr.Number(label="Manual Frequency (Hz)", value=None, precision=6)
                volt_input = gr.Number(label="Manual Amplitude (V)", value=None, precision=6)
                auto_apply_checkbox = gr.Checkbox(label="Auto apply when changed", value=False)
                apply_manual_button = gr.Button("Apply manual inputs", variant="primary")
                gr.HTML("<div style='height:8px'></div>")
                gr.HTML("<h4 style='margin:8px 0 6px 0'>Serial / Arduino</h4>")
                serial_port_input = gr.Textbox(label="Serial port (e.g. /dev/ttyACM0 or COM3)", value="", lines=1)
                ports_out = gr.Textbox(label="Available serial ports", lines=4)
                send_status_out = gr.Textbox(label="Send-to-Arduino status", lines=6)

    # Footer: compact help and legal
    with gr.Row():
        with gr.Column():
            gr.HTML(
                """
                <div style="margin-top:12px;text-align:center;color:var(--muted);font-size:12px">
                  <div>Designed for prototyping. Always verify voltages and connections before connecting hardware.</div>
                  <div style="margin-top:6px">If your API key has been posted publicly, revoke/rotate it now.</div>
                </div>
                """
            )

    # -------------------------
    # (unchanged) Handlers binding
    # -------------------------
    def on_generate(api_key, endpoint, model, prompt):
        gen_cancel_event.clear()
        raw, parsed, code, status = generate_and_parse(api_key.strip(), endpoint.strip(), model.strip() or DEFAULT_MODEL_UI, prompt.strip(), cancel_event=gen_cancel_event)
        return raw, parsed, code, status

    def request_cancel_generation():
        gen_cancel_event.set()
        # Immediately update status so the user sees the cancel request
        return "Generation cancellation requested."

    def on_list_ports():
        return list_all_ports_text()

    def on_send_to_arduino(parsed_json, port):
        send_cancel_event.clear()
        if isinstance(parsed_json, str):
            try:
                parsed = json.loads(parsed_json)
            except Exception:
                return "ERROR: parsed JSON string could not be decoded."
        else:
            parsed = parsed_json
        return send_parsed_to_arduino(parsed, port)

    def request_cancel_send():
        send_cancel_event.set()
        return "Send-to-Arduino cancellation requested."

    def on_manual_apply(hz, volt):
        parsed = build_manual_parsed(hz, volt)
        if parsed.get("frequency_hz") is None or parsed.get("amplitude_v") is None:
            return parsed, manual_parsed_to_code(parsed), "ERROR: Both frequency and amplitude are required.", "Manual input not applied (incomplete inputs)."
        code = manual_parsed_to_code(parsed)
        status = persist_and_set_vars(parsed, persist_path=VARS_FILE_PATH)
        raw_msg = f"Manual input applied: frequency_hz={parsed['frequency_hz']}, amplitude_v={parsed['amplitude_v']}"
        return parsed, code, status, raw_msg

    def on_manual_input_change(hz, volt, auto_apply):
        if auto_apply:
            return on_manual_apply(hz, volt)
        return gr.update(), gr.update(), gr.update(), gr.update()

    # Wire buttons to handlers (order matches outputs declared above)
    gen_button.click(on_generate, inputs=[api_key_input, endpoint_input, model_input, prompt_input], outputs=[raw_out, parsed_out, code_out, status_out])
    cancel_gen_button.click(request_cancel_generation, inputs=[], outputs=[status_out])

    list_ports_button.click(on_list_ports, inputs=[], outputs=[ports_out])
    send_to_arduino_button.click(on_send_to_arduino, inputs=[parsed_out, serial_port_input], outputs=[send_status_out])
    cancel_send_button.click(request_cancel_send, inputs=[], outputs=[send_status_out])

    apply_manual_button.click(on_manual_apply, inputs=[hz_input, volt_input], outputs=[parsed_out, code_out, status_out, raw_out])

    hz_input.change(on_manual_input_change, inputs=[hz_input, volt_input, auto_apply_checkbox], outputs=[parsed_out, code_out, status_out, raw_out])
    volt_input.change(on_manual_input_change, inputs=[hz_input, volt_input, auto_apply_checkbox], outputs=[parsed_out, code_out, status_out, raw_out])

if __name__ == "__main__":
    demo.launch(share=True)

#REVISION ON 11/25/2025 END

* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


## SHOW IF GLOBAL VARIABLES ARE BEING UPDATED

In [8]:
#Test Parameters
print("Hz:", x, ", V:", y)

Hz: 1 , V: 3.3
